In [ ]:
# Run this cell to create widgets
# (and do other things)
%pip install ipywidgets

from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.hashes import SHA256
from cryptography.exceptions import InvalidTag

from IPython.display import display
from ipywidgets import Password, Text

from bs4 import BeautifulSoup
import requests

from base64 import b85encode, b85decode
from zipfile import ZipFile
from io import BytesIO
import os

RENTRY_URL_PREFIX = 'https://rentry.co/'
# RENTRY_URL_PREFIX = 'https://rentry.org/'  # alternative .org domain

class Rentry:
    def __init__(self, rentry_id, rentry_code):
        self.rentry_url = RENTRY_URL_PREFIX
        self.rentry_id = rentry_id
        self.rentry_code = rentry_code
        self.rentry_url_id = self.rentry_url + rentry_id

        self.session = requests.Session()
        self.session.headers['Referer'] = self.rentry_url_id
    
    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.session.close()
        return False

    def _get_token(self):
        response = self.session.get(self.rentry_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.find('input', attrs={'name': 'csrfmiddlewaretoken'})['value']

    def get_raw(self):
        response = self.session.get(self.rentry_url_id + '/raw')
        response.raise_for_status()
        return response.text

    def edit_text(self, text):
        ''' maximum `text` length of 200,000 characters '''
        data = {
            'csrfmiddlewaretoken': self._get_token(),
            'text': text,
            'edit_code': self.rentry_code,
        }

        response = self.session.post(self.rentry_url_id + '/edit', data=data, allow_redirects=False)
        assert response.status_code == 302 and response.headers['Location'] == '/' + self.rentry_id, 'wrong edit_code?'


dirpath = Text(placeholder='In/Out Directory Path')
passwd = Password(placeholder='Password')
display(dirpath)
display(passwd)

In [ ]:
# Run this cell to PULL

key = PBKDF2HMAC(
    algorithm=SHA256(),
    length=48,
    salt=b'\xd1\xaf\xe7\xfag\x89\xa5A\xadt\xc8\xc3\\\xb8\xc6\x8c',
    iterations=1000000,
).derive(passwd.value.encode())
rentry_id, rentry_code, key = key[0:8].hex(), key[8:16].hex(), key[16:48]

with Rentry(rentry_id, rentry_code) as rentry:
    ct = b85decode(rentry.get_raw())  # will fail if password is wrong.
    nonce, ct = ct[:12], ct[12:]

chacha = ChaCha20Poly1305(key)
try:
    pt = chacha.decrypt(nonce, ct, None)
except InvalidTag:
    raise

with ZipFile(BytesIO(pt), 'r') as zipfile:
    zipfile.extractall(dirpath.value)